In [29]:
import os
import pandas as pd

input_path = '/data/wsi/TCTGC50k-labels/TCTGC50k-volume1-labels-path.csv'
output_path = 'onetask_5_labels/all_label.csv'
df = pd.read_csv(input_path)
chosen = ['NILM', 'ASC-US', 'LSIL', 'ASC-H', 'HSIL']
mapping = {'NILM' :0, 'ASC-US' :1, 'LSIL' :2, 'ASC-H' :3, 'HSIL' :4}
df = df[df['wsi_label'].isin(chosen)]
output_df = df[['wsi_id', 'wsi_label']].copy()
output_df['wsi_id'] = output_df['wsi_id'].str.replace('.pt', '')
output_df['wsi_label'] = output_df['wsi_label'].map(mapping)
output_df.to_csv(output_path, index=False)

In [1]:
import os
import pandas as pd
import numpy as np

np.random.seed(0)
label_dir = '/data/wsi/TCTGC10k-labels/9_labels'
input_path = 'all_label.csv'
input_path = os.path.join(label_dir, input_path)
output_path = ['train_label.csv', 'val_label.csv', 'test_label.csv']
output_path = [os.path.join(label_dir, p) for p in output_path]
ratios = [0.8, 0.1, 0.1]
df = pd.read_csv(input_path)
df_shuffle = df.sample(frac=1).reset_index(drop=True)
df_split = [df_shuffle.iloc[:int(len(df_shuffle) * ratios[0])],
            df_shuffle.iloc[int(len(df_shuffle) * ratios[0]):int(len(df_shuffle) * (ratios[0] + ratios[1]))],
            df_shuffle.iloc[int(len(df_shuffle) * (ratios[0] + ratios[1])):]]
for i in range(3):
    df_split[i].to_csv(output_path[i], index=False, header=False)


In [31]:
import os
import pandas as pd
label_dir = 'onetask_5_labels'
label_path = ['train_label.csv', 'val_label.csv', 'test_label.csv']
for i in range(3):
    label_path[i] = os.path.join(label_dir, label_path[i])
    df = pd.read_csv(label_path[i])
    print(df.iloc[:,1].value_counts())
    print('######################{}'.format(len(df)))


0
0    4723
1     983
2     701
3     234
4     198
Name: count, dtype: int64
######################6839
1
0    579
1    118
2    106
3     26
4     25
Name: count, dtype: int64
######################854
0
0    585
1    124
2     94
3     33
4     18
Name: count, dtype: int64
######################854


# 将原始文件抽取需要的信息(wsi_name, wsi_id)到all_label.csv

In [5]:
import pandas as pd
import os

output_path = '/home/huangjialong/projects/BiomedCLIP-PUNCE/datatools/gc10k/9_labels'
label_path = '/data/wsi/TCTGC50k-labels/TCTGC50k-volume1-labels-path.csv'
df = pd.read_csv(label_path)
mapping = {'NILM' :0, 'ASC-US' :1, 'LSIL' :2, 'ASC-H' :3, 'HSIL' :4,\
            'AGC':5, 'BV':6, 'M':7, 'T':8}
# id2class = {v, k for k, v in mapping.items()}
output_file = 'all_label.csv'
df['wsi_label'] = df['wsi_label'].map(mapping)
df['wsi_name'] = df['wsi_name'].str.replace('.pt', '')
df_output = df[['wsi_name', 'wsi_label']].copy()
df_output.to_csv(os.path.join(output_path, output_file), index=False, header=False)

# 根据all_label.csv进行数据集的划分

In [5]:
import pandas as pd
import os

label_dir = '/data/wsi/TCTGC10k-labels/6_labels'
all_label_path = os.path.join(label_dir, 'TCTGC10k-v15-all.csv')
df = pd.read_csv(all_label_path)
print(len(df))
df_shuffled = df.sample(frac=1, random_state=42)

# 根据9:1比例划分
train_ratio = 0.8
df_train = df_shuffled.iloc[:int(len(df_shuffled) * train_ratio)]
df_test = df_shuffled.iloc[int(len(df_shuffled) * train_ratio):]

df_train.to_csv(os.path.join(label_dir, 'TCTGC10k-v15-train.csv'), index=False)
df_test.to_csv(os.path.join(label_dir, 'TCTGC10k-v15-test.csv'), index=False)

print('df_train')
print(df_train['wsi_label'].value_counts())
print('df_test')
print(df_test['wsi_label'].value_counts())

8476
df_train
wsi_label
nilm     4714
ascus     881
lsil      689
asch      228
hsil      185
agc        83
Name: count, dtype: int64
df_test
wsi_label
nilm     1171
ascus     228
lsil      174
asch       58
hsil       54
agc        11
Name: count, dtype: int64


# 挑选出Cancer标注的WSI

In [2]:
import os
import pandas as pd

root_dir = '/data/wsi/TCTGC10k-labels'
input_dir = os.path.join(root_dir, '9_labels', 'TCTGC10k-v15-all.csv')
output_dir = os.path.join(root_dir, '6_labels', 'TCTGC10k-v15-all.csv')
chosen_labels = ['nilm', 'ascus', 'asch', 'lsil', 'hsil', 'agc']
df = pd.read_csv(input_dir)
# 过滤并保存
filtered_df = df[df['wsi_label'].isin(chosen_labels)]
filtered_df.to_csv(output_dir, index=False)
filtered_df['wsi_label'].value_counts()

8476

## 统计训练集 测试集数量

In [7]:
import os
import pandas as pd

root_dir = '/data/wsi/TCTGC50k-labels/6_labels'
files = ['TCTGC50k-v15-all.csv', 'TCTGC50k-v15-train.csv', 'TCTGC50k-v15-test.csv']
for fn in files:
    input_file = os.path.join(root_dir, fn)
    print(input_file)
    df = pd.read_csv(input_file)
    print(df['wsi_label'].value_counts())


/data/wsi/TCTGC50k-labels/6_labels/TCTGC50k-v15-all.csv
wsi_label
nilm     29299
ascus     5300
lsil      4226
asch      1427
hsil      1172
agc        484
Name: count, dtype: int64
/data/wsi/TCTGC50k-labels/6_labels/TCTGC50k-v15-train.csv
wsi_label
nilm     26385
ascus     5033
lsil      4006
asch      1356
hsil      1121
agc        438
Name: count, dtype: int64
/data/wsi/TCTGC50k-labels/6_labels/TCTGC50k-v15-test.csv
wsi_label
nilm     2914
ascus     267
lsil      220
asch       71
hsil       51
agc        46
Name: count, dtype: int64


# 采用分层采用从50k数据集中采样10k的数据集

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split

# 定义输入和输出文件路径
input_path = '/data/wsi/TCTGC50k-labels/6_labels/TCTGC50k-v15-all.csv'
output_path = '/data/wsi/TCTGC10k-labels/6_labels/TCTGC10k-v15-all.csv'

try:
    # 读取CSV文件
    df = pd.read_csv(input_path)

    # 进行分层采样
    # 我们设置 train_size=10000 来获取10000个样本
    sampled_df, _ = train_test_split(
        df,
        train_size=10000,
        stratify=df['wsi_label'],
        random_state=42  # 设置随机种子以保证结果可复现
    )

    # 将采样后的数据保存到新的CSV文件
    sampled_df.to_csv(output_path, index=False)

    print(f"成功从 '{input_path}' 采样10,000条数据，并保存到 '{output_path}'。")
    print("\n采样数据的前5行:")
    print(sampled_df.head())
    print(f"\n原始数据中 'wsi_label' 的分布:\n{df['wsi_label'].value_counts(normalize=True)}")
    print(f"\n采样数据中 'wsi_label' 的分布:\n{sampled_df['wsi_label'].value_counts(normalize=True)}")

except FileNotFoundError:
    print(f"错误: 文件 '{input_path}' 未找到。请检查文件路径是否正确。")
except Exception as e:
    print(f"发生错误: {e}")

成功从 '/data/wsi/TCTGC50k-labels/6_labels/TCTGC50k-v15-all.csv' 采样10,000条数据，并保存到 '/data/wsi/TCTGC10k-labels/6_labels/TCTGC10k-v15-all.csv'。

采样数据的前5行:
          id            wsi_name wsi_label  \
20012  15510            L2013086      nilm   
34595  35727   CX20113910-ASC-US     ascus   
12805   6189            L2015076      lsil   
9283    1099           002908645     ascus   
4300    3778  xCR20017937-ASC-US     ascus   

                                                wsi_path  
20012       /data/wsi/TCTGC50k/TCTGC50k-volume3/L2013086  
34595  /data/wsi/TCTGC50k/TCTGC50k-volume4/CX20113910...  
12805       /data/wsi/TCTGC50k/TCTGC50k-volume2/L2015076  
9283       /data/wsi/TCTGC50k/TCTGC50k-volume2/002908645  
4300   /data/wsi/TCTGC50k/TCTGC50k-volume1/xCR2001793...  

原始数据中 'wsi_label' 的分布:
nilm     0.699127
ascus    0.126468
lsil     0.100840
asch     0.034051
hsil     0.027966
agc      0.011549
Name: wsi_label, dtype: float64

采样数据中 'wsi_label' 的分布:
nilm     0.6991
ascus    0.1265


In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split

# 定义输入文件和输出文件的路径
input_path = '/data/wsi/TCTGC10k-labels/6_labels/TCTGC10k-v15-all.csv'
train_output_path = '/data/wsi/TCTGC10k-labels/6_labels/TCTGC10k-v15-train.csv'
test_output_path = '/data/wsi/TCTGC10k-labels/6_labels/TCTGC10k-v15-test.csv'

try:
    # 读取10k数据集文件
    df = pd.read_csv(input_path)

    # 按照8:2的比例进行分层划分
    # test_size=0.2 表示测试集占20%
    train_df, test_df = train_test_split(
        df,
        test_size=0.2,
        stratify=df['wsi_label'],
        random_state=42  # 确保划分结果可复现
    )

    # 将训练集和测试集分别保存到CSV文件
    train_df.to_csv(train_output_path, index=False)
    test_df.to_csv(test_output_path, index=False)

    print(f"成功将 '{input_path}' 划分为训练集和测试集。")
    print(f"训练集 ({len(train_df)}条) 已保存到: '{train_output_path}'")
    print(f"测试集 ({len(test_df)}条) 已保存到: '{test_output_path}'")
    
    print("\n原始数据集中 'wsi_label' 的分布:")
    print(df['wsi_label'].value_counts(normalize=True))
    print("\n训练集中 'wsi_label' 的分布:")
    print(train_df['wsi_label'].value_counts(normalize=True))
    print("\n测试集中 'wsi_label' 的分布:")
    print(test_df['wsi_label'].value_counts(normalize=True))


except FileNotFoundError:
    print(f"错误: 输入文件 '{input_path}' 未找到。请先确保10k数据集已成功生成。")
except Exception as e:
    print(f"发生错误: {e}")

成功将 '/data/wsi/TCTGC10k-labels/6_labels/TCTGC10k-v15-all.csv' 划分为训练集和测试集。
训练集 (8000条) 已保存到: '/data/wsi/TCTGC10k-labels/6_labels/TCTGC10k-v15-train.csv'
测试集 (2000条) 已保存到: '/data/wsi/TCTGC10k-labels/6_labels/TCTGC10k-v15-test.csv'

原始数据集中 'wsi_label' 的分布:
nilm     0.6991
ascus    0.1265
lsil     0.1008
asch     0.0341
hsil     0.0280
agc      0.0115
Name: wsi_label, dtype: float64

训练集中 'wsi_label' 的分布:
nilm     0.699125
ascus    0.126500
lsil     0.100750
asch     0.034125
hsil     0.028000
agc      0.011500
Name: wsi_label, dtype: float64

测试集中 'wsi_label' 的分布:
nilm     0.6990
ascus    0.1265
lsil     0.1010
asch     0.0340
hsil     0.0280
agc      0.0115
Name: wsi_label, dtype: float64


: 